__Загрузка нужных библиотек__

In [1]:
#для датафрейма
import pandas as pd

#для работы с геоданными
import geopandas as gpd
from shapely.geometry import Point, MultiLineString, shape

#для работы с api-запросами
import json
import requests

#для отслеживания прогресса обновления
from tqdm import tqdm_notebook

from datetime import date

In [2]:
grid = gpd.read_file('moscow_grid_1km.geojson')
grid[['left', 'bottom', 'right', 'top']] = grid['geometry'].bounds
grid.head()

,id,left,top,right,bottom,geometry
0,1,37.36509,55.913094,37.374074,55.908059,"POLYGON ((37.36509 55.91309, 37.37407 55.91309..."
1,2,37.36509,55.908059,37.374074,55.903023,"POLYGON ((37.36509 55.90806, 37.37407 55.90806..."
2,3,37.36509,55.903023,37.374074,55.897987,"POLYGON ((37.36509 55.90302, 37.37407 55.90302..."
3,4,37.36509,55.897987,37.374074,55.892950,"POLYGON ((37.36509 55.89799, 37.37407 55.89799..."
4,5,37.36509,55.892950,37.374074,55.887913,"POLYGON ((37.36509 55.89295, 37.37407 55.89295..."


In [3]:
#пустой датасет для сохранения выгруженных данных
df_all = pd.DataFrame()

In [4]:
for i in tqdm_notebook(grid.index[:5]):
    try:
        #координаты ббокса
        lonmin = str(grid.loc[i,'left'])
        latmin = str(grid.loc[i,'bottom'])
        lonmax = str(grid.loc[i,'right'])
        latmax = str(grid.loc[i,'top'])
        bbox_string = lonmin+ ','+latmin + ','+ lonmax + ',' + latmax
        #api ключ для получения доступа к datamos api
        api_key = '5b5284f3584ee56d9f38ab982fc48ef0'
        #параметры для get-запроса к datamos
        parameters = {'api_key': api_key,
                      'bbox': bbox_string}
        #запрос с параметрами
        response = requests.get("https://apidata.mos.ru/v1/features/60562", params=parameters)
        
        #полученный ответ конвертируем в dataframe
        df = response.json()
        df_pd = pd.DataFrame(df['features'])
        #добавляем выгруженные объекты в общую таблицу
        df_all = df_all.append(df_pd)
    except KeyError:
        pass

In [5]:
df_all.head()

,geometry,properties,type
0,"{'coordinates': [[[37.3717167386076, 55.910395...","{'DatasetId': 60562, 'VersionNumber': 3, 'Rele...",Feature
1,"{'coordinates': [[[37.3699869120991, 55.908806...","{'DatasetId': 60562, 'VersionNumber': 3, 'Rele...",Feature
2,"{'coordinates': [[[37.3703042647069, 55.909498...","{'DatasetId': 60562, 'VersionNumber': 3, 'Rele...",Feature
3,"{'coordinates': [[[37.37282465668, 55.90988053...","{'DatasetId': 60562, 'VersionNumber': 3, 'Rele...",Feature
4,"{'coordinates': [[[37.3730061755201, 55.909847...","{'DatasetId': 60562, 'VersionNumber': 3, 'Rele...",Feature


In [6]:
df_all = df_all.reset_index(drop=True)
df_all['geometry'] = df_all['geometry'].apply(lambda x: shape(x))

In [7]:
df_all.to_csv('datamos_bldg_{}_uncleaned.csv'.format(date.today().strftime("%Y%m%d")), index = False)

In [8]:
#извлечение атрибутивной информации
df_all['properties'][0]['Attributes']

{'UNOM': 3402339,
 'OBJ_TYPE': 'Здание',
 'OnTerritoryOfMoscow': 'да',
 'ADDRESS': 'город Москва, Машкинское шоссе, дом 38, строение 1',
 'SIMPLE_ADDRESS': 'Машкинское шоссе, дом 38, строение 1',
 'P0': None,
 'P1': 'город Москва',
 'P2': '',
 'P3': None,
 'P4': None,
 'P5': 'муниципальный округ Молжаниновский',
 'P6': None,
 'P7': 'Машкинское шоссе',
 'P90': None,
 'P91': None,
 'L1_VALUE': '38',
 'L1_TYPE': 'дом',
 'L2_VALUE': '',
 'L2_TYPE': None,
 'L3_TYPE': 'строение',
 'L3_VALUE': '1',
 'L4_TYPE': None,
 'L4_VALUE': '',
 'L5_TYPE': None,
 'L5_VALUE': '',
 'ADM_AREA': 'Северный административный округ',
 'DISTRICT': 'муниципальный округ Молжаниновский',
 'NREG': 9013195,
 'DREG': '27.01.2006',
 'N_FIAS': '362F6788-8151-4405-85D3-82DBA239F411',
 'D_FIAS': '24.11.2011',
 'KAD_N': [],
 'KAD_ZU': [],
 'KLADR': '77000000000371600',
 'TDOC': 'Распоряжение префектуры АО города Москвы',
 'NDOC': '101',
 'DDOC': '17.01.2006',
 'VID': 'Адрес',
 'ADR_TYPE': 'Официальный',
 'SOSTAD': 'Зарегист

In [9]:
df_all['OBJ_TYPE'] = df_all['properties'].apply(lambda x: x['Attributes']['OBJ_TYPE'])
df_all['ADDRESS'] = df_all['properties'].apply(lambda x: x['Attributes']['ADDRESS'])
df_all['ADM_AREA'] = df_all['properties'].apply(lambda x: x['Attributes']['ADM_AREA'])
df_all['DISTRICT'] = df_all['properties'].apply(lambda x: x['Attributes']['DISTRICT'])
df_all['NREG'] = df_all['properties'].apply(lambda x: x['Attributes']['NREG'])
df_all['DREG'] = df_all['properties'].apply(lambda x: x['Attributes']['DREG'])
df_all['N_FIAS'] = df_all['properties'].apply(lambda x: x['Attributes']['N_FIAS'])
df_all['D_FIAS'] = df_all['properties'].apply(lambda x: x['Attributes']['D_FIAS'])
df_all['KLADR'] = df_all['properties'].apply(lambda x: x['Attributes']['KLADR'])
df_all['TDOC'] = df_all['properties'].apply(lambda x: x['Attributes']['TDOC'])
df_all['NDOC'] = df_all['properties'].apply(lambda x: x['Attributes']['NDOC'])
df_all['DDOC'] = df_all['properties'].apply(lambda x: x['Attributes']['DDOC'])
df_all['VID'] = df_all['properties'].apply(lambda x: x['Attributes']['VID'])
df_all['ADR_TYPE'] = df_all['properties'].apply(lambda x: x['Attributes']['ADR_TYPE'])
df_all['SOSTAD'] = df_all['properties'].apply(lambda x: x['Attributes']['SOSTAD'])
df_all['STATUS'] = df_all['properties'].apply(lambda x: x['Attributes']['STATUS'])
df_all['global_id'] = df_all['properties'].apply(lambda x: x['Attributes']['global_id'])

In [10]:
#удаление дубликатов, которые могли возникнуть при выгрузке
df_pd_final = df_all.drop('properties', axis = 1)
df_pd_final = df_pd_final.drop_duplicates(['global_id'])
df_pd_final.head()

,geometry,type,OBJ_TYPE,ADDRESS,ADM_AREA,DISTRICT,NREG,DREG,N_FIAS,D_FIAS,KLADR,TDOC,NDOC,DDOC,VID,ADR_TYPE,SOSTAD,STATUS,global_id
0,"POLYGON ((37.3717167386076 55.9103956403125, 3...",Feature,Здание,"город Москва, Машкинское шоссе, дом 38, строен...",Северный административный округ,муниципальный округ Молжаниновский,9013195.0,27.01.2006,362F6788-8151-4405-85D3-82DBA239F411,24.11.2011,77000000000371600,Распоряжение префектуры АО города Москвы,101,17.01.2006,Адрес,Официальный,Зарегистрирован в АР,Ожидает внесения в ГКН,646700414
1,"POLYGON ((37.3699869120991 55.9088060283177, 3...",Feature,Здание,"город Москва, Машкинское шоссе, дом 38, строен...",Северный административный округ,муниципальный округ Молжаниновский,9013197.0,27.01.2006,CCA04026-F79F-4DBC-BE58-DFDD8FF9F927,24.11.2011,77000000000371600,Распоряжение префектуры АО города Москвы,101,17.01.2006,Адрес,Официальный,Зарегистрирован в АР,Ожидает внесения в ГКН,648476943
2,"POLYGON ((37.3703042647069 55.9094987126731, 3...",Feature,Здание,"город Москва, Машкинское шоссе, дом 38, строен...",Северный административный округ,муниципальный округ Молжаниновский,9013196.0,27.01.2006,A9EBC196-9B3B-4CFD-BC7C-11D190F96246,24.11.2011,77000000000371600,Распоряжение префектуры АО города Москвы,101,17.01.2006,Адрес,Официальный,Зарегистрирован в АР,Ожидает внесения в ГКН,648649049
3,"POLYGON ((37.37282465668 55.9098805335153, 37....",Feature,Здание,"город Москва, Машкинское шоссе, дом 38, строен...",Северный административный округ,муниципальный округ Молжаниновский,9013198.0,27.01.2006,5A357F74-E49C-4B32-B6FA-4F9C63CDFE5D,24.11.2011,77000000000371600,Распоряжение префектуры АО города Москвы,101,17.01.2006,Адрес,Официальный,Зарегистрирован в АР,Ожидает внесения в ГКН,648884082
4,"POLYGON ((37.3730061755201 55.9098477613325, 3...",Feature,Здание,"город Москва, Машкинское шоссе, дом 38, строен...",Северный административный округ,муниципальный округ Молжаниновский,NaN,,151B9B45-0BA3-4636-A450-ABCD073654F8,24.11.2011,77000000000371600,None,,,Адрес,Основной,-,-,649199021


In [11]:
df_pd_final.to_csv('datamos_bldg_{}.csv'.format(date.today().strftime("%Y%m%d")), index = False)